In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
groq_api_key = os.getenv('GROQ_API_KEY')

gemini_api_key = os.getenv('GEMINI_API_KEY')

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [5]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

c:\Users\SHIVAM GHUGE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


c:\Users\SHIVAM GHUGE\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
import json

with open(r"C:\Users\SHIVAM GHUGE\Downloads\Combined file.xlsx - Sheet1.json",'r',encoding='utf-8') as f:
    investor_data = json.load(f)

In [8]:
from langchain_community.document_loaders import JSONLoader

doc = JSONLoader(r"C:\Users\SHIVAM GHUGE\Downloads\Combined file.xlsx - Sheet1.json",
           jq_schema=".",text_content=False)


In [9]:
docs = doc.load_and_split()

In [10]:
vectordb = FAISS.from_documents(documents=docs,embedding=embeddings,normalize_L2=True)

In [13]:
res = vectordb.similarity_search_with_score(query=q,k=2,fetch_k=10)

In [18]:
retriever = vectordb.as_retriever(search_type="mmr",
                                  search_kwargs={'k':2})

In [25]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct",api_key=groq_api_key)

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate(
    [
    '''
    You have to provide most similar document matching the user query based on the provided context:
    <context>
    {context}
    </context>
    '''
]
)

document_chain = create_stuff_documents_chain(llm=llm,prompt=prompt)

In [27]:
from langchain.chains.retrieval import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever,document_chain)

In [21]:
def load_prompt(startup_profile,investor_profiles):
    prompt = f'''
        You are a skilled business analyst specializing in assessing investor-founder (start-up) compatibility. 
        With a deep understanding of startup ecosystems and investment criteria, your expertise allows you to extract 
        relevant insights and compute compatibility scores effectively.
        Your task is to analyze and process compatibility of founder (start-up) with list of investors.
        The start up details is in json format and investors details are list of json format data.
        Here are the requirements:
            - Founder (start-up) details :
                {startup_profile}

            - Investor details :
                {investor_profiles}
        
        Consider all the important factors when computing the compatibility score for each investor.
        Return a structured output that includes name of investor that matches founder's profile along with their 
        corresponding match score (0-1). Display the ranked results clearly, indicating the highest compatibility first.

        Example output :
            [
                {
                    'AC Venture' : ['match score' : 0.8]
                },
                {
                    'Venture Capital' : ['match score' : 0.7]
                },
                .....
            ]
    '''

    return prompt

In [22]:
import google.generativeai as genai

genai.configure(api_key=gemini_api_key)

In [23]:
gemini_model = genai.GenerativeModel(model_name="gemini-2.0-flash")

In [28]:
founder_data = '''
    {
    Industry : Finance,
    startup stage : Early/Pre-Seed,
    funding required : $100K,
    traction : 15,000 monthly active users (MAUs) within the first 6 months post-launch,
    bussiness model : FinSavvy is a mobile-first personal finance management app designed for millennials and Gen Z.
                      It helps users budget effectively, track expenses, and invest small amounts through micro-investing, all in one platform.
    }
'''

# investor_data = 

In [ ]:
retriever_chain.invoke({
    'input' : founder_data
})

In [ ]:
response = gemini_model.generate_content(
    contents=load_prompt(founder_data,investor_data)
)